<a href="https://colab.research.google.com/github/Hussain1488/Fake-News-Detection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install torchtext kagglehub
# !pip uninstall -y torch torchtext
# !pip install torch==2.3.0 torchtext==0.18.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
import kagglehub
import torchtext
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from torchtext.data.utils import get_tokenizer



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")
print("Path to dataset files:", path)

stop_words = set(stopwords.words('english'))
print(stop_words)

Path to dataset files: /kaggle/input/fake-and-real-news-dataset
{'do', 'your', 'if', 'with', 'between', 'himself', 'doing', 'does', 'under', "we'll", "isn't", "she'll", "he's", "they're", "you're", 'theirs', 'having', "should've", "aren't", 'wouldn', 'into', 'over', 'these', "haven't", 'than', 'shan', "it'd", "wouldn't", 'was', "they've", 'doesn', 'y', 'me', 'once', 'haven', 'ourselves', 'will', 'that', 'who', "it'll", 'few', "didn't", 'then', 'why', 'both', "couldn't", 'any', 'she', 'wasn', 'same', "you'd", 'no', 'all', 'from', 'how', 'as', 'did', "we're", 'too', "you'll", "mightn't", 'when', 'a', 'but', 'their', 'her', 'couldn', 'other', 'yourselves', 'you', 'again', 'weren', 'further', "shouldn't", 'each', 'myself', 'so', 'out', 'didn', 'of', 'm', 'now', 'they', 'not', 't', 'where', "they'd", "hasn't", 'which', "he'd", 'ours', 'own', 'by', "needn't", 'for', "we'd", 'o', "hadn't", 'our', 'it', 'my', 'only', 'his', 'its', 'he', 'down', 'isn', 'some', 'yourself', 'needn', 'here', "it's

# Data Visualization

In [5]:

fake=pd.read_csv(path + '/Fake.csv')
real=pd.read_csv(path + '/True.csv')

for i, text in fake['text'].items():
    filtered_tokens = [token for token in text.split() if token.lower() not in stop_words]
    fake.at[i, 'text'] = filtered_tokens

for i, text in real['text'].items():
    filtered_tokens = [token for token in text.split() if token.lower() not in stop_words]
    real.at[i, 'text'] = filtered_tokens

print(fake.head())
print(real.head())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  [Donald, Trump, wish, Americans, Happy, New, Y...    News   
1  [House, Intelligence, Committee, Chairman, Dev...    News   
2  [Friday,, revealed, former, Milwaukee, Sheriff...    News   
3  [Christmas, day,, Donald, Trump, announced, wo...    News   
4  [Pope, Francis, used, annual, Christmas, Day, ...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  
                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept t

In [6]:
df = pd.concat([fake, real])
df

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,"[Donald, Trump, wish, Americans, Happy, New, Y...",News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,"[House, Intelligence, Committee, Chairman, Dev...",News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"[Friday,, revealed, former, Milwaukee, Sheriff...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"[Christmas, day,, Donald, Trump, announced, wo...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,"[Pope, Francis, used, annual, Christmas, Day, ...",News,"December 25, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,"[BRUSSELS, (Reuters), -, NATO, allies, Tuesday...",worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"[LONDON, (Reuters), -, LexisNexis,, provider, ...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,"[MINSK, (Reuters), -, shadow, disused, Soviet-...",worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,"[MOSCOW, (Reuters), -, Vatican, Secretary, Sta...",worldnews,"August 22, 2017"


In [7]:
df.isnull().sum()


,0
title,0
text,0
subject,0
date,0
